In [1]:
import speech_recognition as srec
from gtts import gTTS
import pyttsx3 as pyt
import os

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

import requests
import json

C:\Users\Alysha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
engine = pyt.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

device = "auto"

In [3]:
# --- AZURE SAILOR 0.5B MODEL ---

model = AutoModelForCausalLM.from_pretrained(
    'sail/Sailor-0.5B-Chat',
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained('sail/Sailor-0.5B-Chat')

In [3]:
# --- LLAMA ---

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [4]:
# --- STT ---

def perintah():
    mendengar = srec.Recognizer()
    with srec.Microphone() as source:
        print('Mendengarkan......')
        suara = mendengar.listen(source, phrase_time_limit=5)
        try:
            print('Diterima.....')
            dengar = mendengar.recognize_google(suara, language='id-ID')
            print(dengar)
        except:
            pass
        return dengar

In [5]:
# --- TTS ---

def ngomong(text):
    voices = engine.getProperty('voices')

    for voice in voices:
        if "MSTTS_V110_idID_Andika" in voice.id:
            engine.setProperty('voice', voice.id)
            break

    # Speak the text
    engine.say(text)
    
    # Wait until speaking is finished
    engine.runAndWait()

In [6]:
def run_va():
    # prompt from user
    Layanan = perintah()

    # messages = [{"role": "user", "content": Layanan}]
    # prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # print(outputs[0]["generated_text"])

    # ngomong(Layanan)

    # print(Layanan)

    system_prompt= 'Jawab dalam bahasa indonesia.'

    messages = [
        {"role": "question", "content": Layanan}
        ]

    # text = tokenizer.apply_chat_template(
    #     messages,
    #     tokenize=False,
    #     add_generation_prompt=True
    # )

    model_inputs = tokenizer(Layanan, return_tensors="pt") 

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=200, 
        repetition_penalty=1.2,

    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)
    ngomong(response)

In [9]:
run_va()

Mendengarkan......
Diterima.....


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Apa ibukota Perancis
?
Perancis dikenal dengan nama Paris, kota terbesar di Eropa dan merupakan pusat ekonomi dunia. Dengan luas wilayah sekitar 1.350 km persegi (520 sq mi),kotanya adalah satu-satunya kota yang berada dalam negara-negara yang mempunyai tiga zona waktu: Greenwich Mean Time London British Summer Time.
Paris is the capital and largest city of France. It has an
